## Import libraries

In [1]:
import numpy as np
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


## Data stuff

### from Multi30k, still not succeed

In [2]:
# from torchtext.datasets import multi30k, Multi30k
# # URL and MD5 replacement for the data (because the server that hosts the data is currently down (as of December 2023))
# URL = {
#   "train": "https://raw.githubusercontent.com/neychev/small_DL_repo/master/datasets/Multi30k/training.tar.gz",
#   "valid": "https://raw.githubusercontent.com/neychev/small_DL_repo/master/datasets/Multi30k/validation.tar.gz",
#   "test": "https://raw.githubusercontent.com/neychev/small_DL_repo/master/datasets/Multi30k/mmt16_task1_test.tar.gz"
# }
# MD5 = {
#   "train": "20140d013d05dd9a72dfde46478663ba05737ce983f478f960c1123c6671be5e",
#   "valid": "a7aa20e9ebd5ba5adce7909498b94410996040857154dab029851af3a866da8c",
#   "test": "6d1ca1dba99e2c5dd54cae1226ff11c2551e6ce63527ebb072a1f70f72a5cd36",
# }
# multi30k.URL["train"] = URL["train"]
# multi30k.URL["valid"] = URL["valid"]
# multi30k.URL["test"] = URL["test"]
# multi30k.MD5["train"] = MD5["train"]
# multi30k.MD5["valid"] = MD5["valid"]
# multi30k.MD5["test"] = MD5["test"]

# print(Multi30k)
# import spacy

# train_data, valid_data, test_data = Multi30k(
#   root=".data",
#   split=("train", "valid", "test"),
#   language_pair=("de", "en")
# )
# print(test_data)
# print(list(test_data))


### from dummy

In [3]:
train_src_sentences = [
  "Selamat pagi!",
  "Apa kabar?",
  "Saya lapar.",
  "Tolong bantu saya.",
  "Terima kasih banyak.",
  "Saya suka musik.",
  "Ini buku saya.",
  "Berapa umurmu?",
  "Hujan turun.",
  "Tolong berikan saya air.",
  "Kamu cantik.",
  "Saya mencintai keluarga saya.",
  "Apa pekerjaanmu?",
  "Saya ingin belajar bahasa Inggris.",
  "Di mana kamar mandi?",
  "Apa warna favoritmu?",
  "Saya pergi ke sekolah setiap hari.",
  "Bunga ini indah.",
  "Tolong tunggu sebentar.",
  "Saya suka makan nasi goreng.",
  "Ayah saya seorang dokter.",
  "Apa nama hewan ini?",
  "Saya senang bertemu denganmu.",
  "Ini mobil baru saya.",
  "Anjing itu lucu.",
  "Saya suka berenang di pantai.",
  "Apa cita-citamu?",
  "Saya belajar bahasa Indonesia.",
  "Hari ini cuaca panas.",
  "Saya suka menonton film.",
  "Saya punya dua saudara.",
  "Di mana toko buku terdekat?",
  "Apa yang kamu lakukan kemarin?",
  "Saya ingin memesan makanan.",
  "Kamu pandai.",
  "Apa rencanamu untuk liburan?",
  "Mobil itu sangat cepat.",
  "Saya suka minum teh hangat.",
  "Anak-anak bermain di taman.",
  "Apa nama kota ini?",
  "Saya suka sekali dengan film ini.",
  "Apa yang kamu pelajari di sekolah?",
  "Hari ini adalah ulang tahun saya.",
  "Saya ingin menjadi penulis.",
  "Saya suka pergi ke pantai pada malam hari.",
  "Tolong jangan lupa membayar tagihan listrik.",
  "Anak itu pintar sekali.",
  "Saya senang bisa membantu.",
  "Buku ini sangat tebal.",
  "Kamu memiliki hobi apa?",
  "Apa yang kamu pikirkan tentang film itu?",
  "Pohon-pohon itu tinggi sekali.",
  "Saya ingin mencoba makanan khas Indonesia.",
  "Di sini sangat ramai.",
  "Tolong tutup pintu.",
  "Apa makanan favoritmu?",
  "Rumah ini sangat bersih.",
  "Saya suka bermain game komputer.",
  "Tiket pesawat sudah dibeli.",
  "Mobil itu rusak.",
  "Saya senang bisa belajar dari kesalahan.",
  "Pemandangan di gunung ini sangat indah.",
  "Hewan-hewan itu liar.",
  "Tolong beri tahu saya arah yang benar.",
  "Saya suka menulis puisi.",
  "Apakah kamu suka olahraga?",
  "Tolong kirim email ini segera.",
  "Saya ingin berlibur ke pantai.",
  "Kamu punya hewan peliharaan?",
  "Pakaian ini terlalu kecil.",
  "Apa acara favoritmu di televisi?",
  "Saya suka berjalan-jalan di taman.",
  "Anak-anak itu sedang bermain sepak bola.",
  "Saya akan pergi ke pasar besok.",
  "Apa pendapatmu tentang musik klasik?",
  "Saya ingin memesan kamar untuk dua malam.",
  "Tolong beri tahu saya cara ke stasiun kereta.",
  "Saya belajar memasak dari ibu.",
  "Kami akan pergi ke pantai bersama-sama.",
  "Apakah kamu suka pergi ke konser musik?",
  "Saya merasa bangga dengan prestasi anak-anak saya."
]
train_trg_sentences = [
  "Good morning!",
  "How are you?",
  "I am hungry.",
  "Please help me.",
  "Thank you very much.",
  "I like music.",
  "This is my book.",
  "How old are you?",
  "It's raining.",
  "Please give me water.",
  "You are beautiful.",
  "I love my family.",
  "What is your job?",
  "I want to learn English.",
  "Where is the bathroom?",
  "What is your favorite color?",
  "I go to school every day.",
  "These flowers are beautiful.",
  "Please wait a moment.",
  "I like to eat fried rice.",
  "My father is a doctor.",
  "What is the name of this animal?",
  "I am happy to meet you.",
  "This is my new car.",
  "That dog is cute.",
  "I like to swim at the beach.",
  "What is your ambition?",
  "I am learning Indonesian.",
  "Today, the weather is hot.",
  "I like to watch movies.",
  "I have two siblings.",
  "Where is the nearest bookstore?",
  "What did you do yesterday?",
  "I would like to order food.",
  "You are clever.",
  "What are your plans for the holiday?",
  "That car is very fast.",
  "I like to drink hot tea.",
  "The children are playing in the park.",
  "What is the name of this city?",
  "I really like this movie.",
  "What do you study in school?",
  "Today is my birthday.",
  "I want to be a writer.",
  "I like to go to the beach at night.",
  "Please don't forget to pay the electricity bill.",
  "That child is very smart.",
  "I am happy to help.",
  "This book is very thick.",
  "What is your hobby?",
  "What do you think about that movie?",
  "Those trees are very tall.",
  "I want to try Indonesian cuisine.",
  "It is very crowded here.",
  "Please close the door.",
  "What is your favorite food?",
  "This house is very clean.",
  "I like to play computer games.",
  "The plane tickets have been purchased.",
  "That car is broken.",
  "I am happy to learn from mistakes.",
  "The scenery on this mountain is very beautiful.",
  "Those animals are wild.",
  "Please tell me the right direction.",
  "I like to write poetry.",
  "Do you like sports?",
  "Please send this email immediately.",
  "I want to vacation at the beach.",
  "Do you have any pets?",
  "This clothes are too small.",
  "What is your favorite TV show?",
  "I like to take a stroll in the park.",
  "The children are playing soccer.",
  "I will go to the market tomorrow.",
  "What is your opinion about classical music?",
  "I would like to book a room for two nights.",
  "Please tell me the way to the train station.",
  "I learn to cook from my mother.",
  "We will go to the beach together.",
  "Do you like to go to music concerts?",
  "I feel proud of my children's achievements."
]
valid_src_sentences = [
  "Saya suka menonton film animasi.",
  "Apa rencanamu untuk akhir pekan ini?",
  "Rumah itu terlihat sangat nyaman.",
  "Saya ingin mencoba makanan Korea.",
  "Buku ini sangat menarik.",
  "Tolong beritahu saya nomor teleponnya.",
  "Kami akan mengadakan pesta ulang tahun besok.",
  "Apakah kamu punya rencana perjalanan musim panas ini?",
  "Saya suka bermain game papan dengan teman-teman.",
]
valid_trg_sentences = [
  "I like to watch animated movies.",
  "What are your plans for this weekend?",
  "That house looks very comfortable.",
  "I want to try Korean food.",
  "This book is very interesting.",
  "Please tell me his/her phone number.",
  "We will have a birthday party tomorrow.",
  "Do you have any plans for this summer vacation?",
  "I like to play board games with friends."
]
test_src_sentences = [
  "Tolong ambilkan saya secangkir kopi.",
  "Saya suka menghabiskan waktu di perpustakaan.",
  "Hewan-hewan di kebun binatang terlihat bahagia.",
  "Saya akan mengunjungi kakek nenek saya akhir pekan ini.",
  "Tolong beri tahu saya cara menggunakan mesin cuci.",
  "Pesta itu akan dimulai pukul delapan malam.",
  "Anak-anak sedang bermain di taman bermain.",
  "Apakah kamu suka memasak masakan Thailand?",
  "Saya ingin belajar bermain gitar.",
  "Hari ini adalah hari libur nasional."
]
test_trg_sentences = [
  "Please get me a cup of coffee.",
  "I like to spend time in the library.",
  "The animals at the zoo look happy.",
  "I will visit my grandparents this weekend.",
  "Please tell me how to use the washing machine.",
  "The party will start at eight in the evening.",
  "The children are playing at the playground.",
  "Do you like to cook Thai dishes?",
  "I want to learn to play the guitar.",
  "Today is a national holiday."
]


## Data preprocessing

In [4]:
import re
from collections import Counter
from torch.nn.utils.rnn import pad_sequence

def build_vocab(text_corpus, min_freq=1) :
  # Tokenize the text into words
  tokenized_text = [["<sos>"] + re.findall(r"\b\w+\b", sentence.lower()) + ["<eos>"] for sentence in text_corpus]
  # print("tokenized_text", tokenized_text)

  # Flatten the list of tokenized words
  flat_tokens = [token for sentence_tokens in tokenized_text for token in sentence_tokens]
  # print("flat_tokens", flat_tokens)

  # Count the frequency of each token
  token_freq = Counter(flat_tokens)
  # print("token_freq", token_freq)

  # Filter tokens based on minimum frequency
  tokens = [token for token, freq in token_freq.items() if freq >= min_freq]
  # print("tokens", tokens)

  # Create a vocabulary mapping from token to index
  vocab = {token: idx for idx, token in enumerate(tokens)}
  # print("vocab", vocab)

  # Create a reverse mapping from index to token
  idx_to_token = {idx: token for token, idx in vocab.items()}
  # print("idx_to_token", idx_to_token)

  return vocab, idx_to_token, tokenized_text

train_src_vocab, train_src_itos, train_src_tokenized_sentences = build_vocab(train_src_sentences)
train_trg_vocab, train_trg_itos, train_trg_tokenized_sentences = build_vocab(train_trg_sentences)
valid_src_vocab, valid_src_itos, valid_src_tokenized_sentences = build_vocab(valid_src_sentences)
valid_trg_vocab, valid_trg_itos, valid_trg_tokenized_sentences = build_vocab(valid_trg_sentences)
test_src_vocab, test_src_itos, test_src_tokenized_sentences = build_vocab(test_src_sentences)
test_trg_vocab, test_trg_itos, test_trg_tokenized_sentences = build_vocab(test_trg_sentences)

train_src_data = [torch.tensor([train_src_vocab[word] for word in sentence]) for sentence in train_src_tokenized_sentences]
train_trg_data = [torch.tensor([train_trg_vocab[word] for word in sentence]) for sentence in train_trg_tokenized_sentences]
valid_src_data = [torch.tensor([valid_src_vocab[word] for word in sentence]) for sentence in valid_src_tokenized_sentences]
valid_trg_data = [torch.tensor([valid_trg_vocab[word] for word in sentence]) for sentence in valid_trg_tokenized_sentences]
test_src_data = [torch.tensor([test_src_vocab[word] for word in sentence]) for sentence in test_src_tokenized_sentences]
test_trg_data = [torch.tensor([test_trg_vocab[word] for word in sentence]) for sentence in test_trg_tokenized_sentences]
train_src_data = pad_sequence(train_src_data, batch_first=True, padding_value=-1)
train_trg_data = pad_sequence(train_trg_data, batch_first=True, padding_value=-1)
valid_src_data = pad_sequence(valid_src_data, batch_first=True, padding_value=-1)
valid_trg_data = pad_sequence(valid_trg_data, batch_first=True, padding_value=-1)
test_src_data = pad_sequence(test_src_data, batch_first=True, padding_value=-1)
test_trg_data = pad_sequence(test_trg_data, batch_first=True, padding_value=-1)

## Preparing Tensor Dataset and DataLoader

In [5]:

from torch.utils.data import DataLoader, TensorDataset

# Create TensorDataset(s)
train_dataset = TensorDataset(train_src_data, train_trg_data)
valid_dataset = TensorDataset(valid_src_data, valid_trg_data)
test_dataset = TensorDataset(test_src_data, test_trg_data)

# Set batch size
batch_size = 32

# Create DataLoader(s)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

for i, batch in enumerate(train_loader) :
  print(i, batch[0].shape, batch[1].shape)

0 torch.Size([32, 10]) torch.Size([32, 12])
1 torch.Size([32, 10]) torch.Size([32, 12])
2 torch.Size([17, 10]) torch.Size([17, 12])


## Model Definitions

### Encoder

In [23]:
class Encoder(nn.Module) :
  def __init__(self, vocab_len, emb_dim, enc_hidden_dim, num_layers=1, dropout=.5) -> None :
    super().__init__()
    self.embedding = nn.Embedding(vocab_len, emb_dim)
    self.rnn = nn.LSTM(
      input_size=emb_dim,
      hidden_size=enc_hidden_dim,
      num_layers=num_layers,
      dropout=dropout
    )
    self.dropout = nn.Dropout(p=dropout)

  def forward(self, input_batch) :
    embedded = self.dropout(self.embedding(input_batch))
    outputs, hidden = self.rnn(embedded)
    return outputs, hidden

### Attention (Additive)

In [22]:
class Attention(nn.Module) :
  def __init__(self, enc_hidden_dim, dec_hidden_dim) -> None :
    super().__init__()
    # The input dim will be the concatenation of
    # enc_hidden_dim (hidden) and dec_hidden_dim (encoder_outputs)
    self.attn_hidden_vector = nn.Linear(
      in_features=enc_hidden_dim+dec_hidden_dim,
      out_features=dec_hidden_dim,
      # bias=False
    )

    # We need source len number of values for n batch as the dimension
    # of the alignment or attention weights. The attn_hidden_vector will have the
    # dimension of [source len, batch size, decoder hidden dim]

    # If we set the output dim of this Linear layer to 1, then the effective
    # output dimension will be [source len, batch size]
    self.attn_scoring_fn = nn.Linear(dec_hidden_dim, 1, bias=False)
  
  def forward(self, hidden, encoder_outputs) :
    # hidden dimension = [1, batch size, decoder hidden dim]
    src_len = encoder_outputs.shape[0]

    # We need to calculate the attn_hidden for each source token.
    # Instead of repeating this using a loop, we can duplicate
    # hidden by src_len number of times and perform the operations.
    hidden = hidden.repeat(src_len, 1, 1)

    # Calculate attention hidden values (tanh(W_c[E_o + D_h^(t-1)]))
    attn_hidden = torch.tanh(self.attn_hidden_vector(torch.cat((hidden, encoder_outputs), dim=2)))

    # Calculate the attention scoring vector. Remove 3rd dimension.
    attn_scoring_vector = self.attn_scoring_fn(attn_hidden).squeeze(2)

    # The attn_scoring_vector now has dimension of [source len, batch size]
    # Since we need to calculate the softmax per record in the batch, we will
    # have to switch the dimension to [batch size, source len]
    attn_scoring_vector = attn_scoring_vector.permute(1, 0)

    # Softmax function for normalizing the weights to probability distribution
    # Further dot product with encoder outputs will be done in OneStepDecoder
    return F.softmax(attn_scoring_vector, dim=1)

### OneStepDecoder
OneStepDecoder will handle most of the data flow in the Decoder class, such as applying the embedding function to the data output, calculating the alignment or attention weights, predicting the token outputs

In [21]:
class OneStepDecoder(nn.Module) :
  def __init__(self, input_output_dim, emb_dim, enc_hidden_dim, dec_hidden_dim, attn, dropout=.5) -> None :
    super().__init__()
    self.output_dim = input_output_dim
    self.attn = attn
    self.embedding = nn.Embedding(input_output_dim, embedding_dim=emb_dim)

    # Add the enc_hidden_dim and emb_dim
    self.rnn = nn.LSTM(
      input_size=enc_hidden_dim+emb_dim,
      hidden_size=dec_hidden_dim
    )
    # Combine all the features for better prediction
    self.fc = nn.Linear(
      in_features=enc_hidden_dim+dec_hidden_dim+emb_dim,
      out_features=input_output_dim
    )
    self.dropout = nn.Dropout(p=dropout)
  
  def forward(self, input, hidden, encoder_outputs) :
    # Add the source len dimension
    input = input.unsqueeze(0)
    embedded = self.dropout(self.embedding(input))

    # Calculate the alignment or attention weights
    s_t = self.attn(hidden, encoder_outputs).unsqueeze(1)

    # We need to perform the batch-wise dot product.
    # Hence we need to shift the batch size dimension to the front.
    encoder_outputs = encoder_outputs.permute(1, 0, 2)

    # Use PyTorch's bmm function to calculate the attention matrix c_t
    c_t = torch.bmm(s_t, encoder_outputs)
    # Revert the batch dimension
    c_t = c_t.permute(1, 0, 2)

    # Concatenate the previous output with c_t
    rnn_input = torch.cat((embedded, c_t), dim=2)

    output, hidden = self.rnn(rnn_input, hidden)

    # Remove the token length dimension and pass them to the Linear layer
    predicted_token = self.fc(torch.cat((output.squeeze(0), c_t.squeeze(0), embedded.squeeze(0)), dim=1))

    return predicted_token, hidden, s_t.squeeze(1)

### Decoder

Here is the **Decoder** module. We need to pass the `encoder_outputs` to the `one_step_decoder`'s forward method.

Also notice that for the first loop of the hidden state will be the encoder's hidden state.

In [9]:
class Decoder(nn.Module) :
  def __init__(self, one_step_dec, device) -> None :
    super().__init__()
    self.one_step_dec = one_step_dec
    self.device = device

  def forward(self, target, encoder_outputs, hidden, teacher_forcing_ratio=.5) :
    batch_size = target.shape[1]
    trg_len = target.shape[0]
    trg_vocab_size = self.one_step_dec.output_dim

    outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
    input = target[0, :]

    for t in range(1, trg_len) :
      # Pass the encoder_outputs. For the first timestep, the hidden state
      # comes from the encoder model
      output, hidden, s_t = self.one_step_dec(input, hidden, encoder_outputs)
      outputs[t] = output

      teacher_force = random.random() < teacher_forcing_ratio
      top1 = output.argmax(1)

      input = target[t] if teacher_force else top1

    return outputs

### EncoderDecoder

A wrapper class for whole encoder and decoder class

In [10]:
class EncoderDecoder(nn.Module) :
  def __init__(self, enc, dec) -> None :
    super().__init__()
    self.enc = enc
    self.dec = dec
  
  def forward(self, source, target, teacher_forcing_ratio=.5) :
    encoder_outputs, hidden = self.enc(source)
    return self.dec(target, encoder_outputs, hidden, teacher_forcing_ratio)

## Modelling

In [26]:
def create_model(source_vocab, target_vocab) :
  # Define the required dimensions and hyperparameters
  embedding_dim = 256
  hidden_dim = 1024
  dropout = .5

  # Instantiate the models
  attention = Attention(hidden_dim, hidden_dim)
  encoder = Encoder(len(source_vocab), embedding_dim, hidden_dim, dropout=dropout)
  one_step_decoder = OneStepDecoder(len(target_vocab), embedding_dim, hidden_dim, hidden_dim, attention, dropout=dropout)
  decoder = Decoder(one_step_decoder, device)

  model = EncoderDecoder(encoder, decoder)
  # model = model.to(device)

  # Define the optimizer
  optimizer = optim.Adam(model.parameters())

  # Make sure the CrossEntropyLoss ignores the padding tokens.
  TARGET_PAD_IDX = -1
  criterion = nn.CrossEntropyLoss(ignore_index=TARGET_PAD_IDX)

  return model, optimizer, criterion
model, optimizer, criterion = create_model(train_src_vocab, train_trg_vocab)
print(model, optimizer, criterion)

EncoderDecoder(
  (enc): Encoder(
    (embedding): Embedding(180, 256)
    (rnn): LSTM(256, 1024, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (dec): Decoder(
    (one_step_dec): OneStepDecoder(
      (attn): Attention(
        (attn_hidden_vector): Linear(in_features=2048, out_features=1024, bias=True)
        (attn_scoring_fn): Linear(in_features=1024, out_features=1, bias=False)
      )
      (embedding): Embedding(184, 256)
      (rnn): LSTM(1280, 1024)
      (fc): Linear(in_features=2304, out_features=184, bias=True)
      (dropout): Dropout(p=0.5, inplace=False)
    )
  )
) Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.001
    maximize: False
    weight_decay: 0
) CrossEntropyLoss()


## Training script

In [36]:
def train(train_loader, valid_loader, source_vocab, target_vocab, epochs=10) :
  model, optimizer, criterion = create_model(source_vocab, target_vocab)

  clip = 1

  for epoch in range(epochs) :
    progress_bar = tqdm(total=len(train_loader.dataset), bar_format="{l_bar}{bar:10}{r_bar}{bar:-10b}", unit=" data", ncols=200)

    train_losses = []
    # Set model to training mode
    model.train()

    # Loop through the training batch
    for i, batch in enumerate(train_loader) :
      # Get the source and target tokens
      src = batch[0].permute(1,0)
      trg = batch[1].permute(1,0)
      print(src.shape, trg.shape)

      optimizer.zero_grad()

      # Forward propagation
      output = model(src, trg)
      # Reshape the output
      output_dim = output.shape[-1]
      # Discard the first token as this will always be 0
      output = output[1:].view(-1, output_dim)
      # Discard the <sos> token from target
      trg = trg[1:].view(-1)
      # Calculate the loss
      loss = criterion(output, trg)
      # Back propagation
      loss.backward()
      #TODO: Gradient clipping for stability
      # Apply gradient descent to model
      optimizer.step()

      train_losses.append(loss.item())
      progress_bar.set_postfix(
        epoch=f" {epoch}, train loss= {round(sum(train_losses)/len(train_losses), 4)}", refresh=True
      )
      progress_bar.update()
    
    # Evaluate model
    with torch.no_grad() :
      # Set the model to evaluation mode
      model.eval()

      valid_losses = []

      # Loop through the validation batch
      for i, batch in enumerate(valid_loader) :
        src = batch[0]
        trg = batch[1]

        # Forward propagation
        output = model(src, trg, teacher_forcing_ratio=0)
        # Reshape the output
        output_dim = output.shape[-1]
        # Discard the first token as this will always be 0
        output = output[1:].view(-1, output_dim)
        # Discard the <sos> token from target
        trg = trg[1:].view(-1)
        # Calculate the loss
        loss = criterion(output, trg)
        
        valid_losses.append(loss.item())
      
    progress_bar.set_postfix(
      epoch=f" {epoch}, train loss= {round(sum(train_losses)/len(train_losses), 4)}, val loss= {round(sum(valid_losses)/len(valid_losses), 4)}", refresh=True
    )
    progress_bar.close()
  return model

## bismillah

In [37]:
CUDA_LAUNCH_BLOCKING="1"
model = train(train_loader, valid_loader, train_src_vocab, train_trg_vocab, epochs=10)
print(model)

/home/ahmadnaufalhakim/.local/lib/python3.8/site-packages/torch/nn/modules/rnn.py:71: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "





torch.Size([10, 32]) torch.Size([12, 32])


IndexError: index out of range in self